In [2]:
import pydot
import os
import json
import requests

import numpy as np
import tensorflow as tf
import keras_tuner as kt

from pathlib import Path
from time import strftime

ModuleNotFoundError: No module named 'tensorflow'

# Using Keras to load the dataset

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

In [5]:
fashion_mnist

((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

In [7]:
X_train, y_train =  X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid =  X_train_full[-5000:], y_train_full[-5000:]

In [8]:
X_train.shape

(55000, 28, 28)

In [9]:
X_train.dtype

dtype('uint8')

In [10]:
# we'll scale the pixel intensities down to the 0-1 range
X_train, X_valid, X_test = X_train / 225., X_valid / 225., X_test / 225.

In [11]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [12]:
class_names[y_train[0]]

'Ankle boot'

# Creating the model using the sequential API

In [13]:
tf.random.set_seed(73) # tf.keras.utils.set_random_seed() sets the random seeds TensorFlow, Python and Numpy

# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Input(shape=[28, 28]))
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(300, activation="relu"))
# model.add(tf.keras.layers.Dense(100, activation="relu"))
# model.add(tf.keras.layers.Dense(10, activation="softmax"))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

2023-04-28 21:33:30.825242: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-28 21:33:30.850913: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-04-28 21:33:30.850931: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-04-28 21:33:30.851173: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neur

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [15]:
# tf.keras.utils.plot_model(model)

Keras ensures the name a layers is globally unique, even across models. This makes it possible to merge models easily without getting name conflicts.

In [16]:
model.layers

In [17]:
hidden1 = model.layers[1]
hidden1.name

'dense'

In [18]:
model.get_layer('dense') is hidden1

True

In [19]:
weights, biases = hidden1.get_weights()
weights

array([[-0.00625843,  0.01640897,  0.05140327, ..., -0.00060768,
        -0.01147474, -0.05572239],
       [-0.07035636, -0.0329636 , -0.00621657, ..., -0.03180042,
        -0.02941287, -0.0701891 ],
       [-0.03781017, -0.01585966, -0.04156978, ...,  0.06098467,
        -0.07037002,  0.0163684 ],
       ...,
       [ 0.06787972,  0.00012163,  0.06985936, ...,  0.04520102,
         0.01807659, -0.05321223],
       [-0.04921868, -0.06580366,  0.06535976, ...,  0.04199433,
        -0.02728266,  0.05598585],
       [-0.051929  , -0.04107234,  0.0257173 , ..., -0.03419239,
         0.05619784,  0.06763771]], dtype=float32)

In [20]:
weights.shape

(784, 300)

In [21]:
biases

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [22]:
biases.shape

(300,)

In [23]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

## Model Checkpoint

In [24]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_checkpoints", save_weights_only=True, save_best_only=True)

## Early Stopping

In [25]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

## TensorBoard

In [26]:
def get_run_logdir(root_logdir="my_logs"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

In [27]:
run_logdir = get_run_logdir()

In [28]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))

2023-04-28 21:33:40.365161: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-28 21:33:40.365189: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-04-28 21:33:40.365237: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1664] Profiler found 1 GPUs
2023-04-28 21:33:40.365752: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2023-04-28 21:33:40.371739: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.
2023-04-28 21:33:40.371876: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1798] CUPTI activity buffer flushed


In [47]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])

Epoch 1/30
 245/1719 [===>..........................] - ETA: 1s - loss: 0.1977 - accuracy: 0.9311

2023-04-28 17:05:19.696313: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-04-28 17:05:19.696338: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-04-28 17:05:19.833835: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2023-04-28 17:05:19.834070: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1798] CUPTI activity buffer flushed
2023-04-28 17:05:19.841352: I tensorflow/core/profiler/backends/gpu/cupti_collector.cc:522]  GpuTracer has collected 0 callback api events and 0 activity events. 
2023-04-28 17:05:19.841875: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.
2023-04-28 17:05:19.842017: I tensorflow/core/profiler/rpc/client/save_profile.cc:164] Collecting XSpace to repository: my_logs/run_2023_04_28_17_05_15/plugins/profile/2023_04_28_17_05_19/raha-MS-7C37.xplane.pb


1719/1719 [==============================] - 3s 2ms/step - loss: 0.2045 - accuracy: 0.9275 - val_loss: 0.2967 - val_accuracy: 0.8928
Epoch 2/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.2025 - accuracy: 0.9275 - val_loss: 0.3152 - val_accuracy: 0.8904
Epoch 3/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.1984 - accuracy: 0.9292 - val_loss: 0.3036 - val_accuracy: 0.8908
Epoch 4/30
1719/1719 [==============================] - 3s 1ms/step - loss: 0.1952 - accuracy: 0.9297 - val_loss: 0.3076 - val_accuracy: 0.8896
Epoch 5/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.1926 - accuracy: 0.9313 - val_loss: 0.3100 - val_accuracy: 0.8894
Epoch 6/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.1882 - accuracy: 0.9322 - val_loss: 0.3032 - val_accuracy: 0.8916
Epoch 7/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.1853 - accuracy: 0.9343 - val_loss: 0.3411 - val_accuracy: 0.8826
Epo

In [25]:
model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version

In [26]:
model.save(model_path, save_format="tf")

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


In [27]:
!saved_model_cli show --dir my_mnist_model/0001

2023-04-28 16:22:59.036164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 16:22:59.547762: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-28 16:22:59.547802: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-28 16:22:59.547807: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [28]:
!saved_model_cli show --dir my_mnist_model/0001 --tag_set serve

2023-04-28 16:23:00.860790: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 16:23:01.373561: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-28 16:23:01.373601: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-28 16:23:01.373607: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

__saved_model_init_op --> an initialization function

serving_default --> defult serving function

In [29]:
!saved_model_cli show --dir my_mnist_model/0001 --tag_set serve --signature_def serving_default

2023-04-28 16:23:02.647540: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 16:23:03.156354: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-28 16:23:03.156393: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-28 16:23:03.156399: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

# TensorFlow Serving

Run install-tf-server.sh 

In [30]:
%pip install -q -U tensorflow-serving-api

Note: you may need to restart the kernel to use updated packages.


In [31]:
os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

In [32]:
%%bash --bg
tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=my_mnist_model --model_base_path="${MODEL_DIR}" > my_server.log 2>&1

It is now waiting for gRPC and REST requests, respectively, on ports 8500 and 8501

# Querying TF Serving through the REST API

In [33]:
X_new = X_test[:6] # pretend we have 3 new digit images to classify
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [35]:
server_url = "http://127.0.0.1:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [36]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.02, 0.  , 0.97],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.36, 0.  , 0.03, 0.  , 0.  , 0.  , 0.6 , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [37]:
y_test[:6]

array([9, 2, 1, 1, 6, 1], dtype=uint8)

# TensorBoard

In [49]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 24553), started 0:03:48 ago. (Use '!kill 24553' to kill it.)

In [51]:
%tensorboard --logdir=./my_logs

# Fine-Tuning

In [29]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    
    return model

In [31]:
random_search_tuner = kt.RandomSearch(build_model, 
                                      objective="val_accuracy", 
                                      max_trials=5, 
                                      overwrite=True, 
                                      directory="my_fashion_mnist",
                                      project_name="my_rnd_search",
                                      seed=73)
random_search_tuner.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Trial 5 Complete [00h 00m 12s]
val_accuracy: 0.7121999859809875

Best val_accuracy So Far: 0.8808000087738037
Total elapsed time: 00h 02m 11s
INFO:tensorflow:Oracle triggered exit


In [ ]:
class MyClassificationHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)
    
    def fit(self, hp, model, X, y, **kwargs):
        if hp.Boolean("normalize"):
            norm_layer = tf.keras.layers.Normalization()
            X = norm_layer(X)
        return model.fit(X, y, **kwargs)

In [ ]:
# max_epochs controls the max number of epochs that the best model will be trained for
# hyperband_iterations is the number the whole process is repeated
hyperband_tuner = kt.Hyperband(MyClassificationHyperModel(), objective="val_accuracy", seed=73, max_epochs=10, 
                               factor=3, hyperband_iterations=2, overwrite=True, directory="my_fashion_mnist",
                               project_name="hyperband")

In [ ]:
root_logdir = Path(hyperband_tuner.project_dir) / "tensorboard"
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
hyperband_tuner.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), 
                       callbacks=[early_stopping_cb, tensorboard_cb])

In [32]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

In [35]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values # best hyperparameter values

{'n_hidden': 4,
 'n_neurons': 50,
 'learning_rate': 0.0009824542588158123,
 'optimizer': 'adam'}

In [37]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 2 summary
Hyperparameters:
n_hidden: 4
n_neurons: 50
learning_rate: 0.0009824542588158123
optimizer: adam
Score: 0.8808000087738037


In [38]:
best_trial.metrics.get_last_value("val_accuracy")

0.8808000087738037